In [1]:
from google.colab import files
import os

print("Upload model configuration file in pickle or joblib format.")
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  _, ext = os.path.splitext(fn)

  # If the file is not a pickle or joblib file, delete it
  if ext.lower() != '.pkl' and ext.lower() != '.joblib':
      print(f"Deleting non-model configuration file: {fn}")
      os.remove(fn)
  else:
      print(f"Model configuration file ok: {fn}")

Upload model configuration file in pickle or joblib format.


Saving RandomForestClassifier.pkl to RandomForestClassifier.pkl
User uploaded file "RandomForestClassifier.pkl" with length 687063 bytes
Model configuration file ok: RandomForestClassifier.pkl


In [2]:
pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 830.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.3 MB/s eta 0:00:00


In [3]:
import py7zr
print("Upload archive file of images with masks (10 mb max) in 7z format.")

# Upload the files
uploaded = files.upload()

# Create a new folder
root_folder = 'Images/'
os.makedirs(root_folder, exist_ok=True)

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  _, ext = os.path.splitext(fn)

  if ext.lower() != '.7z':
      print(f"Deleting non-archive file: {fn}")
      os.remove(fn)
  else:
      print(f"7z file ok: {fn}")
      new_path = os.path.join(root_folder, fn)
      os.rename(fn, new_path)

      # Extract the 7z file
      with py7zr.SevenZipFile(new_path, mode='r') as z:
          z.extractall(path=root_folder)

      print(f"Extracted archive file: {fn}")

Upload archive file of images with masks (10 mb max) in 7z format.


Saving test image.7z to test image.7z
User uploaded file "test image.7z" with length 1880869 bytes
7z file ok: test image.7z
Extracted archive file: test image.7z


In [4]:
your_directory_path = '/content/Images'
folder_names = []
for folder_name in os.listdir(your_directory_path):
    if os.path.isdir(os.path.join(your_directory_path, folder_name)):
        folder_names.append(folder_name)

In [5]:
from skimage.segmentation import slic, mark_boundaries
from skimage.util import img_as_float
from skimage import io, color, morphology,img_as_ubyte
from skimage.transform import resize
from skimage.feature import graycomatrix,graycoprops
from skimage import exposure
from skimage.measure import regionprops

from scipy.spatial import Delaunay
from scipy.linalg import eigh
from scipy.stats import entropy,skew

import networkx as nx
import cv2
import math
import pywt
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
#from google.colab import drive
#drive.mount('/content/gdrive')



from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import pickle
import fnmatch

import warnings
# Ignore pandas warnings
warnings.filterwarnings('ignore')

In [6]:
def generate_graph(img_path, mask_path):
  image = img_as_float(io.imread(img_path))
  image = cv2.resize(image, (750, 750))  # Resize the image to 750x750 pixels
  mask = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
  mask = cv2.resize(mask, (750, 750))

  # Apply SLIC with masking and extract (approximately) the supplied number of segments
  segments = slic(image, n_segments=20, sigma=18, mask=mask)
  middle_pixels = np.zeros((np.max(segments) + 1, 2))

  # Calculate a non-border valid pixel for each segment
  for label in np.unique(segments):
      if label == 0:  # Skip segment 0
          continue

      mask = segments == label
      valid_pixels = np.where(mask & mask)  # Select valid pixels (mask == 1)
      if len(valid_pixels[0]) > 0:  # Check if there are valid pixels in the region
          valid_rows, valid_cols = valid_pixels
          distances_to_border = np.minimum(
              np.minimum(valid_rows, mask.shape[0] - 1 - valid_rows),
              np.minimum(valid_cols, mask.shape[1] - 1 - valid_cols)
          )
          non_border_indices = np.where(distances_to_border >= 20)[0]  # Minimum distance from border increased to 20
          if len(non_border_indices) > 0:
              random_index = np.random.choice(non_border_indices)  # Choose a random index
              random_pixel = (valid_cols[random_index], valid_rows[random_index])  # Note the change in ordering
              middle_pixels[label] = random_pixel

  # Calculate Delaunay triangulation based on the middle pixels (excluding segment 0)
  valid_middle_pixels = middle_pixels[middle_pixels[:, 0] != 0]  # Exclude segment 0
  triangulation = Delaunay(valid_middle_pixels)

  G = nx.Graph()
  for simplex in triangulation.simplices:
      G.add_edge(simplex[0], simplex[1])
      G.add_edge(simplex[1], simplex[2])
      G.add_edge(simplex[2], simplex[0])
  return G

In [7]:
def process(img_path, mask_path):
  #graph domain
  G = generate_graph(img_path, mask_path)
  ge = global_efficiency(G)
  local_efficiencies = local_efficiency(G)#for node in graph.nodes():
  local_clustering_coeffs = local_clustering_coefficient(G)#for node in graph.nodes():
  nodal_strengths = nodal_strength(G)#for node in graph.nodes()
  nodal_betweenness = nodal_betweenness_centrality(G)
  sorted_nodes_nodal_betweenness = sorted(nodal_betweenness.items(), key=lambda x: x[0])
  closeness_centralities = closeness_centrality(G)
  sorted_nodes_closeness_centralities = sorted(closeness_centralities.items(), key=lambda x: x[0])
  eccentricities = nx.eccentricity(G)
  sorted_nodes_eccentricities = sorted(eccentricities.items(), key=lambda x: x[0])
  char_path_length = characteristic_path_length(G)
  global_clustering_coeff = global_clustering_coefficient(local_clustering_coeffs)
  graph_density_value = graph_density(G)
  global_assortativity_value = global_assortativity(G)

  #freq domain
  f_h = calc_f_hat(G)
  ffd_energy,ffd_power,ffd_spectral_entropy,ffd_amplitude = finish_frequency_domain(G)

  #geometry domain
  geo_feats = calc_geo_feats(mask_path)

  # COLOR FEATURES
  image = cv2.imread(img_path)
  image = cv2.resize(image, (750, 750))#TODO: optimize to reuse existing
  mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
  mask = cv2.resize(mask, (750, 750))#TODO: optimize to reuse existing
  # Mask the grayscale image using the binary mask
  masked_image = cv2.bitwise_and(image, image, mask=mask)

  # Calculate the mean, variance, and standard deviation of the masked image
  mean_value = np.mean(masked_image)
  variance_value = np.var(masked_image)
  std_deviation = np.sqrt(variance_value)

  # Load the image
  image = cv2.imread(img_path)
  image = cv2.resize(image, (750, 750))#TODO: optimize to reuse existing

  # Split the image into color channels (BGR channels for OpenCV)
  blue_channel, green_channel, red_channel = cv2.split(image)

  # Convert the image to HSV color space
  hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  hue_channel, sat_channel, val_channel = cv2.split(hsv_image)

  # Convert the image to CIE Lab color space
  lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
  lab_l_channel, lab_a_channel, lab_b_channel = cv2.split(lab_image)

  # Convert the image to CIE Luv color space
  luv_image = cv2.cvtColor(image, cv2.COLOR_BGR2Luv)
  luv_l_channel, luv_u_channel, luv_v_channel = cv2.split(luv_image)

  # Calculate statistics for each channel in different color spaces
  # HSV Color Space
  avg_hue, var_hue, std_dev_hue, min_hue, max_hue, skew_hue = calculate_stats(hue_channel)
  avg_sat, var_sat, std_dev_sat, min_sat, max_sat, skew_sat = calculate_stats(sat_channel)
  avg_val, var_val, std_dev_val, min_val, max_val, skew_val = calculate_stats(val_channel)

  # CIE Lab Color Space
  avg_lab_l, var_lab_l, std_dev_lab_l, min_lab_l, max_lab_l, skew_lab_l = calculate_stats(lab_l_channel)
  avg_lab_a, var_lab_a, std_dev_lab_a, min_lab_a, max_lab_a, skew_lab_a = calculate_stats(lab_a_channel)
  avg_lab_b, var_lab_b, std_dev_lab_b, min_lab_b, max_lab_b, skew_lab_b = calculate_stats(lab_b_channel)

  # CIE Luv Color Space
  avg_luv_l, var_luv_l, std_dev_luv_l, min_luv_l, max_luv_l, skew_luv_l = calculate_stats(luv_l_channel)
  avg_luv_u, var_luv_u, std_dev_luv_u, min_luv_u, max_luv_u, skew_luv_u = calculate_stats(luv_u_channel)
  avg_luv_v, var_luv_v, std_dev_luv_v, min_luv_v, max_luv_v, skew_luv_v = calculate_stats(luv_v_channel)

  # Calculate statistics for each channel
  avg_blue, var_blue, std_dev_blue, min_blue, max_blue, skew_blue = calculate_stats(blue_channel)
  avg_green, var_green, std_dev_green, min_green, max_green, skew_green = calculate_stats(green_channel)
  avg_red, var_red, std_dev_red, min_red, max_red, skew_red = calculate_stats(red_channel)
  # COLOR FEATURES END

  #Hausdorff
  image = cv2.imread(img_path)
  image = cv2.resize(image, (750, 750))#TODO: optimize to reuse existing
  r_channel, g_channel, b_channel_rgb = cv2.split(image)

  luv_image = cv2.cvtColor(image.astype(np.float32), cv2.COLOR_BGR2LUV)
  l_channel_luv, u_channel, v_channel_luv = cv2.split(luv_image)

  lab_image = cv2.cvtColor(image.astype(np.float32), cv2.COLOR_BGR2LAB)
  l_channel_lab, a_channel, b_channel_lab = cv2.split(lab_image)

  hsv_image = cv2.cvtColor(image.astype(np.float32), cv2.COLOR_BGR2HSV)
  h_channel, s_channel, v_channel_hsv = cv2.split(hsv_image)

  Hausdorff_rgb_r = Get_Hausdorff_dim(r_channel)
  Hausdorff_rgb_g =Get_Hausdorff_dim(g_channel)
  Hausdorff_rgb_b =Get_Hausdorff_dim(b_channel_rgb)

  Hausdorff_luv_l = Get_Hausdorff_dim(l_channel_luv)
  Hausdorff_luv_u = Get_Hausdorff_dim(u_channel)
  Hausdorff_luv_v = Get_Hausdorff_dim(v_channel_luv)

  Hausdorff_lab_l = Get_Hausdorff_dim(l_channel_lab)
  Hausdorff_lab_a = Get_Hausdorff_dim(a_channel)
  Hausdorff_lab_b = Get_Hausdorff_dim(b_channel_lab)

  Hausdorff_hsv_h = Get_Hausdorff_dim(h_channel)
  Hausdorff_hsv_s = Get_Hausdorff_dim(s_channel)
  Hausdorffk_hsv_v = Get_Hausdorff_dim(v_channel_hsv)
  #Hausdorff end

  #DWT
  image = cv2.imread(img_path)
  image = cv2.resize(image, (750, 750))#TODO: optimize to reuse existing
  r_channel, g_channel, b_channel_rgb = cv2.split(image)

  luv_image = cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
  l_channel_luv, u_channel, v_channel_luv = cv2.split(luv_image)

  lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
  l_channel_lab, a_channel, b_channel_lab = cv2.split(lab_image)

  hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  h_channel, s_channel, v_channel_hsv = cv2.split(hsv_image)

  tmpd1 = Get_waves(r_channel, g_channel, b_channel_rgb,'rgb_')
  tmpd2 = Get_waves(l_channel_luv, u_channel, v_channel_luv,'luv_')
  tmpd3 = Get_waves(l_channel_lab, a_channel, b_channel_lab,'lab_')
  tmpd4 = Get_waves(h_channel, s_channel, v_channel_hsv,'hsv')
  #DWT end

  #GLCM
  Haralick_rgb = Calc_GLCMs(['R_','G_','B_'],r_channel,g_channel,b_channel_rgb)
  Haralick_luv = Calc_GLCMs(['L_','U_','V_'],l_channel_luv,u_channel,v_channel_luv)
  Haralick_lab_l = Calc_GLCMs(['L_','A_','B_'],l_channel_lab,a_channel,b_channel_lab)
  Haralick_hsv_h = Calc_GLCMs(['H_','S_','V_'],h_channel,s_channel,v_channel_hsv)
  #GLCM end
  geo_data = {}


  geo_data = {'geo_area_1': [geo_feats[0]],
          'geo_perimeter_1': [geo_feats[1]],
          'geo_equivalent_diameter_1': [geo_feats[2]],
          'geo_compactness_1': [geo_feats[3]],
          'geo_circularity_1': [geo_feats[4]],
          'geo_solidity_1': [geo_feats[5]],
          'geo_rectangularity_1': [geo_feats[6]],
          'geo_aspect_ratio_1': [geo_feats[7]],
          'geo_eccentricity_1': [geo_feats[8]],
          }

  color_stats_data = {
      'avg_hue': avg_hue, 'var_hue': var_hue, 'std_dev_hue': std_dev_hue, 'min_hue': min_hue, 'max_hue': max_hue, 'skew_hue': skew_hue,
      'avg_sat': avg_sat, 'var_sat': var_sat, 'std_dev_sat': std_dev_sat, 'min_sat': min_sat, 'max_sat': max_sat, 'skew_sat': skew_sat,
      'avg_val': avg_val, 'var_val': var_val, 'std_dev_val': std_dev_val, 'min_val': min_val, 'max_val': max_val, 'skew_val': skew_val,
      'avg_lab_l': avg_lab_l, 'var_lab_l': var_lab_l, 'std_dev_lab_l': std_dev_lab_l, 'min_lab_l': min_lab_l, 'max_lab_l': max_lab_l, 'skew_lab_l': skew_lab_l,
      'avg_lab_a': avg_lab_a, 'var_lab_a': var_lab_a, 'std_dev_lab_a': std_dev_lab_a, 'min_lab_a': min_lab_a, 'max_lab_a': max_lab_a, 'skew_lab_a': skew_lab_a,
      'avg_lab_b': avg_lab_b, 'var_lab_b': var_lab_b, 'std_dev_lab_b': std_dev_lab_b, 'min_lab_b': min_lab_b, 'max_lab_b': max_lab_b, 'skew_lab_b': skew_lab_b,
      'avg_luv_l': avg_luv_l, 'var_luv_l': var_luv_l, 'std_dev_luv_l': std_dev_luv_l, 'min_luv_l': min_luv_l, 'max_luv_l': max_luv_l, 'skew_luv_l': skew_luv_l,
      'avg_luv_u': avg_luv_u, 'var_luv_u': var_luv_u, 'std_dev_luv_u': std_dev_luv_u, 'min_luv_u': min_luv_u, 'max_luv_u': max_luv_u, 'skew_luv_u': skew_luv_u,
      'avg_luv_v': avg_luv_v, 'var_luv_v': var_luv_v, 'std_dev_luv_v': std_dev_luv_v, 'min_luv_v': min_luv_v, 'max_luv_v': max_luv_v, 'skew_luv_v': skew_luv_v,
      'avg_blue': avg_blue, 'var_blue': var_blue, 'std_dev_blue': std_dev_blue, 'min_blue': min_blue, 'max_blue': max_blue, 'skew_blue': skew_blue,
      'avg_green': avg_green, 'var_green': var_green, 'std_dev_green': std_dev_green, 'min_green': min_green, 'max_green': max_green, 'skew_green': skew_green,
      'avg_red': avg_red, 'var_red': var_red, 'std_dev_red': std_dev_red, 'min_red': min_red, 'max_red': max_red, 'skew_red': skew_red
  }

  hausdorff_data = {
      'Hausdorff_rgb_r': Hausdorff_rgb_r,
      'Hausdorff_rgb_g': Hausdorff_rgb_g,
      'Hausdorff_rgb_b': Hausdorff_rgb_b,
      'Hausdorff_luv_l': Hausdorff_luv_l,
      'Hausdorff_luv_u': Hausdorff_luv_u,
      'Hausdorff_luv_v': Hausdorff_luv_v,
      'Hausdorff_lab_l': Hausdorff_lab_l,
      'Hausdorff_lab_a': Hausdorff_lab_a,
      'Hausdorff_lab_b': Hausdorff_lab_b,
      'Hausdorff_hsv_h': Hausdorff_hsv_h,
      'Hausdorff_hsv_s': Hausdorff_hsv_s,
      'Hausdorffk_hsv_v': Hausdorffk_hsv_v
  }

  # Create a simple dataframe with one column and one value
  data = {'global_efficiency': [ge],
          'char_path_length': [char_path_length],
          'global_clustering_coeff': [global_clustering_coeff],
          'graph_density_value': [graph_density_value],
          'global_assortativity_value': [global_assortativity_value],
          #
          'frequency_domain_energy': [ffd_energy],
          'frequency_domain_power': [ffd_power],
          'frequency_domain_entropy': [ffd_spectral_entropy],
          'frequency_domain_amplitude': [ffd_amplitude],
          }

  data.update(geo_data)
  data.update(color_stats_data)
  data.update(hausdorff_data)
  data.update(tmpd1)
  data.update(tmpd2)
  data.update(tmpd3)
  data.update(tmpd4)

  data.update(Haralick_rgb)
  data.update(Haralick_luv)
  data.update(Haralick_lab_l)
  data.update(Haralick_hsv_h)

  df = pd.DataFrame(data)

  # Append each value as a new column
  for i, value in enumerate(local_efficiencies, start=1):
      df[f'local_efficiency_node_{i}'] = [value]

  # Append each value as a new column
  for i, value in enumerate(local_clustering_coeffs, start=1):
      df[f'local_clustering_coef_node_{i}'] = [value]

  # Append each value as a new column
  for i, value in enumerate(nodal_strengths, start=1):
      df[f'strength_node_{i}'] = [value]

  # Append each value as a new column
  for i, value in sorted_nodes_nodal_betweenness:
      df[f'nodal_betweenness_node_{i+1}'] = [value]

  # Append each value as a new column
  for i, value in sorted_nodes_closeness_centralities:
      df[f'closeness_centrality_node_{i+1}'] = [value]

  # Append each value as a new column
  for i, value in sorted_nodes_eccentricities:
      df[f'eccentricity_node_{i+1}'] = [value]

  for i, value in enumerate(f_h.tolist(),start=1):
      df[f'f_hat{i}'] = [value]

  # Write the dataframe object into CSV file
  df.to_csv('test.csv', index=None, header=True)
  return df

# GLOBAL METHODS

In [19]:
def characteristic_path_length(graph):
    total_path_length = 0
    num_pairs = 0

    for node_i in graph.nodes():
        for node_j in graph.nodes():
            if node_i != node_j:
                shortest_path_length = nx.shortest_path_length(graph, source=node_i, target=node_j)
                total_path_length += shortest_path_length
                num_pairs += 1

    characteristic_length = total_path_length / num_pairs
    return characteristic_length

In [20]:
# Calculate global efficiency for the graph
def global_efficiency(graph):
    total_efficiency = 0.0
    num_pairs = 0

    for source_node in graph.nodes():
        for target_node in graph.nodes():
            if source_node != target_node:
                try:
                    sp_length = nx.shortest_path_length(graph, source=source_node, target=target_node)
                    total_efficiency += 1.0 / sp_length
                    num_pairs += 1
                except nx.NetworkXNoPath:
                    # Nodes are not connected
                    pass

    if num_pairs > 0:
        global_efficiency = total_efficiency / num_pairs
        return global_efficiency
    else:
        return 0.0

In [21]:
def global_clustering_coefficient(local_clustering_coeffs):
    num_nodes = len(local_clustering_coeffs)
    global_coefficient = sum(local_clustering_coeffs) / num_nodes
    return global_coefficient

In [22]:
# Calculate graph density for the unweighted graph
def graph_density(graph):
    num_edges = graph.number_of_edges()
    num_nodes = graph.number_of_nodes()
    max_possible_edges = num_nodes * (num_nodes - 1)

    density = (2 * num_edges) / max_possible_edges
    return density

In [23]:
def global_assortativity(graph):
    assortativity = nx.degree_assortativity_coefficient(graph)
    return assortativity

# LOCAL METHODS

In [13]:
# Calculate local efficiency for each node
def local_efficiency(graph):
    local_efficiencies = []

    # Calculate global efficiency for the entire graph
    global_efficiency_value = global_efficiency(graph)

    for node in graph.nodes():
        neighbors = list(graph.neighbors(node))

        # Create subgraph of neighbors
        subgraph = graph.subgraph(neighbors)

        # Calculate the global efficiency for the subgraph
        subgraph_efficiency = global_efficiency(subgraph)

        # Calculate local efficiency as a ratio of subgraph_efficiency to global_efficiency_value
        if global_efficiency_value != 0:
            local_efficiency_value = subgraph_efficiency / global_efficiency_value
        else:
            local_efficiency_value = 0.0

        local_efficiencies.append(local_efficiency_value)

    return local_efficiencies

In [14]:
def local_clustering_coefficient(graph):
    local_clustering_coefficients = []

    for node in graph.nodes():
        neighbors = list(graph.neighbors(node))
        degree = graph.degree(node)

        # Count the number of edges between neighbors
        edges_within_neighbors = sum(1 for u, v in graph.subgraph(neighbors).edges() if u in neighbors and v in neighbors)

        # Calculate local clustering coefficient for the node
        if degree > 1:
            lcc = (2 * edges_within_neighbors) / (degree * (degree - 1))
        else:
            lcc = 0.0

        local_clustering_coefficients.append(lcc)

    return local_clustering_coefficients

In [15]:
def nodal_strength(graph):
    nodal_strengths = [graph.degree(node) for node in graph.nodes()]
    return nodal_strengths

In [16]:
# Calculate nodal betweenness centrality for each node
def nodal_betweenness_centrality(graph):
    nodal_betweenness = nx.betweenness_centrality(graph)
    return nodal_betweenness

In [17]:
def closeness_centrality(graph):
    closeness_centralities = nx.closeness_centrality(graph)
    return closeness_centralities

In [18]:
# Calculate eccentricity for each node
def eccentricity(graph):
    eccentricities = nx.eccentricity(graph)
    return eccentricities

# Features in the Frequency Domain

In [24]:
def create_adjacency_matrix(graph):
    num_nodes = graph.number_of_nodes()
    adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for edge in graph.edges():
        node_i, node_j = edge
        adjacency_matrix[node_i, node_j] = 1
        adjacency_matrix[node_j, node_i] = 1  # For undirected graph

    return adjacency_matrix

In [25]:
def calc_f_hat(G):
  adjacency_matrix = create_adjacency_matrix(G)
  #Calculate the degree of vertices
  degrees = dict(G.degree())
  # Construct the degree matrix
  degree_matrix = np.diag(list(degrees.values()))
  # Calculate shortest path distances between connected vertices
  shortest_paths = nx.shortest_path_length(G)
  L = degree_matrix - adjacency_matrix
  # Calculate eigenvalues (Λ) and eigenvectors (U)
  eigenvalues, eigenvectors = np.linalg.eigh(L)
  return np.dot(eigenvalues, eigenvectors)

In [26]:
def finish_frequency_domain(G):
  # Compute the Laplacian matrix
  adjacency_matrix = create_adjacency_matrix(G)
  degree_matrix = np.diag(adjacency_matrix.sum(axis=1))
  laplacian_matrix = degree_matrix - adjacency_matrix

  # Compute the eigenvalues
  eigenvalues = eigh(laplacian_matrix, eigvals_only=True)

  # Normalize the eigenvalues for a probability distribution
  eigenvalues /= eigenvalues.sum()

  # Energy is the sum of the square of the eigenvalues
  energy = np.sum(np.square(eigenvalues))
  # Power is the energy divided by the size of the eigenvalues
  power = energy / len(eigenvalues)
  # Entropy is a statistical measure of randomness
  spectral_entropy = entropy(eigenvalues)
  # Amplitude is the square root of the energy
  amplitude = np.sqrt(energy)
  return energy,power,spectral_entropy,amplitude

# Geometry-based graph signals

In [27]:
# Create a function to calculate contour properties
def calculate_contour_properties(contour,binary_image):
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)

    if area > 0:
        equivalent_diameter = np.sqrt(4 * area / np.pi)
        compactness = (perimeter ** 2) / (4 * np.pi * area)
        circularity = (4 * np.pi * area) / (perimeter ** 2)
        ellipse = cv2.fitEllipse(contour)

        # The second element of the tuple returned by cv2.fitEllipse() contains the lengths of the major and minor axes
        major_axis = max(ellipse[1])
        minor_axis = min(ellipse[1])

        # Calculate the eccentricity
        eccentricity = np.sqrt(1 - (minor_axis / major_axis)**2) if major_axis > 0 else 0
    else:
        equivalent_diameter = 0
        compactness = 0
        circularity = 0
        eccentricity = 0

    hull = cv2.convexHull(contour)
    solidity = (area / cv2.contourArea(hull)) if cv2.contourArea(hull) > 0 else 0

    x, y, w, h = cv2.boundingRect(contour)
    rectangularity = area / (w * h)
    aspect_ratio = w / h

    return area, perimeter, equivalent_diameter, compactness, circularity, solidity, rectangularity, aspect_ratio, eccentricity

In [28]:
def calc_geo_feats(mask_path):
  image = cv2.imread(mask_path)
  image = cv2.resize(image, (750, 750))#TODO: optimize to reuse existing
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Threshold or segment the image as needed
  ret, binary_image = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

  # Find contours
  contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contour_image = image.copy()

  geo_feats_list = []
  for contour in contours:
      if cv2.contourArea(contour) > 100:
        area, perimeter, equivalent_diameter, compactness, circularity, solidity, rectangularity, aspect_ratio, eccentricity = calculate_contour_properties(contour,binary_image)
        geo_feats_list.append(area)
        geo_feats_list.append(perimeter)
        geo_feats_list.append(equivalent_diameter)
        geo_feats_list.append(compactness)
        geo_feats_list.append(circularity)
        geo_feats_list.append(solidity)
        geo_feats_list.append(rectangularity)
        geo_feats_list.append(aspect_ratio)
        geo_feats_list.append(eccentricity)
        break#image should contain only 1 contour

  return geo_feats_list

In [29]:
# Function to calculate statistics for a given channel
def calculate_stats(channel):
    average = np.mean(channel)
    variance = np.var(channel)
    std_deviation = np.std(channel)
    min_val = np.min(channel)
    max_val = np.max(channel)
    skewness = skew(channel, axis=None)
    return average, variance, std_deviation, min_val, max_val, skewness

# Hausdorff

In [30]:
def Get_Hausdorff_dim(channel):
  pixels = []
  image = channel
  for i in range(image.shape[0]):
      for j in range(image.shape[1]):
          if image[i, j] > 0:
              pixels.append((i, j))

  Lx = image.shape[1]
  Ly = image.shape[0]
  pixels = np.array(pixels)
  scales = np.logspace(0.01, 1, num=10, endpoint=False, base=2)
  Ns = []

  for scale in scales:
      H, edges = np.histogramdd(pixels, bins=(np.arange(0, Lx, scale), np.arange(0, Ly, scale)))
      Ns.append(np.sum(H > 0))

  coeffs = np.polyfit(np.log(scales), np.log(Ns), 1)
  return -coeffs[0]

# DWT

In [31]:
def Get_waves(channel1, channel2, channel3,prefix):
  dict_entry = {}
  # Loop through the channels and calculate energy and entropy for each sub-band at each level
  channel_counter = 1
  for channel in [channel1, channel2, channel3]:
    counter = 1
    coeffs = pywt.wavedec2(channel, 'haar', level=3)
    first_10_subbands = coeffs[1:10]

    dict_entry[prefix+f'channel_{channel_counter}'+f'_a_energy_{counter}'] = calculate_energy_numpy(coeffs[0])
    dict_entry[prefix+f'channel_{channel_counter}'+f'_a_entropy_{counter}'] = calculate_entropy_scipy(coeffs[0])

    for subband in first_10_subbands:
      horizontal_details_level, vertical_details_level, diagonal_details_level = subband

      horizontal_energy = calculate_energy_numpy(horizontal_details_level)
      vertical_energy = calculate_energy_numpy(vertical_details_level)
      diagonal_energy = calculate_energy_numpy(diagonal_details_level)

      horizontal_entropy_value = calculate_entropy_scipy(horizontal_details_level)
      vertical_entropy_value = calculate_entropy_scipy(vertical_details_level)
      diagonal_entropy_value = calculate_entropy_scipy(diagonal_details_level)

      #rgb_channel_x_energy_x
      dict_entry[prefix+f'channel_{channel_counter}'+f'_h_energy_{counter}'] = horizontal_energy
      dict_entry[prefix+f'channel_{channel_counter}'+f'_v_energy_{counter}'] = vertical_energy
      dict_entry[prefix+f'channel_{channel_counter}'+f'_d_energy_{counter}'] = diagonal_energy

      dict_entry[prefix+f'channel_{channel_counter}'+f'_h_entropy_{counter}'] = horizontal_entropy_value
      dict_entry[prefix+f'channel_{channel_counter}'+f'_v_entropy_{counter}'] = vertical_entropy_value
      dict_entry[prefix+f'channel_{channel_counter}'+f'_d_entropy_{counter}'] = diagonal_entropy_value

      counter = counter + 1
    channel_counter = channel_counter + 1

  return dict_entry

In [32]:
def calculate_entropy(sub_band):
    # Square the sub-band
    squared = np.square(sub_band)

    # Calculate the log of the squared sub-band, handling the case where the value is 0
    log_squared = np.where(squared != 0, np.log(squared), 0)

    # Multiply the squared sub-band by the log of the squared sub-band
    product = squared * log_squared

    # Sum all the values and divide by the number of elements to get the entropy
    entropy_c = np.sum(product) / np.size(sub_band)

    return entropy_c

In [33]:
def calculate_energy(sub_band):
    # Square the sub-band
    squared = np.square(sub_band)

    # Sum all the values and divide by the number of elements
    mean_square = np.sum(squared) / np.size(sub_band)

    # Take the square root to get the energy
    energy_c = np.sqrt(mean_square)

    return energy_c

In [34]:
def calculate_entropy_scipy(sub_band):
    # Flatten the sub-band to 1D and calculate the probability distribution
    p = np.square(sub_band).flatten()
    p /= np.sum(p)

    # Calculate the entropy using scipy
    entropy_value = entropy(p)

    return entropy_value

def calculate_energy_numpy(sub_band):
    # Calculate the energy (Euclidean norm) using numpy
    energy_c = np.linalg.norm(sub_band)

    return energy_c

# (GLCMs)

In [35]:
def calculate_variance(glcm):
    N = glcm.shape[0]
    I, J = np.ogrid[0:N, 0:N]
    mu = np.apply_over_axes(np.sum, I * glcm, axes=(0, -1))
    variance = np.apply_over_axes(np.sum, (I - mu) ** 2 * glcm, axes=(0, -1))
    return variance

def calculate_sum_entropy(glcm):
    num_levels, num_levels, num_distances, num_angles = glcm.shape
    sum_entropy = 0
    for d in range(num_distances):
        for a in range(num_angles):
            glcm_da = glcm[:, :, d, a]
            p_x_plus_y = np.array([np.sum(glcm_da[i:i+num_levels, j:j+num_levels]) for i in range(num_levels) for j in range(num_levels)])
            sum_entropy += -np.nansum(p_x_plus_y * np.log2(p_x_plus_y + (p_x_plus_y == 0)))
    return sum_entropy

# Calculate IMC1 (Information Measure of Correlation 1)
def calculate_imc1pi(glcm):
    N = glcm.shape[0]
    I, J = np.ogrid[0:N, 0:N]
    hx = np.sum(glcm * np.log2(glcm + np.finfo(float).eps), axis=0)
    hy = np.sum(glcm * np.log2(glcm + np.finfo(float).eps), axis=1)
    hxy1 = -np.sum(glcm * np.log2(glcm + np.finfo(float).eps))
    hxy2 = -np.sum(glcm * np.log2(glcm + np.finfo(float).eps))
    imc1 = (hxy1 - hxy2) / np.max(hx, hy)
    return imc1

# Calculate IMC2 (Information Measure of Correlation 2)
def calculate_imc2pi(glcm):
    N = glcm.shape[0]
    I, J = np.ogrid[0:N, 0:N]
    hx = np.sum(glcm * np.log2(glcm + np.finfo(float).eps), axis=0)
    hy = np.sum(glcm * np.log2(glcm + np.finfo(float).eps), axis=1)
    hxy1 = -np.sum(glcm * np.log2(glcm + np.finfo(float).eps))
    hxy2 = -np.sum(glcm * np.log2(glcm + np.finfo(float).eps))
    imc2 = (1 - np.exp(-2 * (hxy2 - hxy1))) ** 0.5
    return imc2

def calculate_difference_entropy(glcm):
    diff_entropy = 0
    num_directions = glcm.shape[3]

    for direction in range(num_directions):
        glcm_direction = glcm[:, :, 0, direction]
        diff_entropy += -np.sum(glcm_direction * np.log(glcm_direction + (glcm_direction == 0)))

    return diff_entropy
###
# Calculate IMC1 (Information Measure of Correlation 1)
def calculate_imc1(glcm):
    HXY = -np.sum(glcm * np.log(glcm + (glcm == 0)))
    HX = -np.sum(np.sum(glcm, axis=0) * np.log(np.sum(glcm, axis=0) + (np.sum(glcm, axis=0) == 0)))
    HY = -np.sum(np.sum(glcm, axis=2) * np.log(np.sum(glcm, axis=2) + (np.sum(glcm, axis=2) == 0)))
    IMC1 = (HXY - HX - HY) / max(HX, HY)
    return IMC1

# Calculate IMC2 (Information Measure of Correlation 2)
def calculate_imc2(glcm):
    HXY = -np.sum(glcm * np.log(glcm + (glcm == 0)))
    HX = -np.sum(np.sum(glcm, axis=0) * np.log(np.sum(glcm, axis=0) + (np.sum(glcm, axis=0) == 0)))
    HY = -np.sum(np.sum(glcm, axis=2) * np.log(np.sum(glcm, axis=2) + (np.sum(glcm, axis=2) == 0)))
    IMC2 = np.sqrt(1 - np.exp(-2 * (HXY - HX - HY)))
    return IMC2

###
def calculate_difference_entropy_pi(glcm):
    N = glcm.shape[0]
    I, J = np.ogrid[0:N, 0:N]
    diff = np.abs(I - J)
    p_diff = np.sum(glcm * (diff == diff[:, None]), axis=0)
    diff_entropy = -np.sum(p_diff * np.log2(p_diff + np.finfo(float).eps))
    return diff_entropy

# Calculate MCC (Maximal Correlation Coefficient)
def calculate_mcc(glcm):
    C = np.outer(np.sum(glcm, axis=2), np.sum(glcm, axis=0))
    U, S, Vt = np.linalg.svd(glcm)
    MCC = np.max(S) / np.sqrt(np.sum(C * C))
    return MCC

In [36]:
def Calc_GLCMs(channel_names,red_channel,green_channel,blue_channel):
  fullname = ''.join(channel_names)
  glcm_dict = {}

  # Quantize the intensities into 16 levels
  red_quantized = img_as_ubyte(red_channel) // 16
  green_quantized = img_as_ubyte(green_channel) // 16
  blue_quantized = img_as_ubyte(blue_channel) // 16

  # List of color channels
  channels = [red_quantized, green_quantized, blue_quantized]

  for channel, name in zip(channels, channel_names):
      # Calculate the GLCM
      glcm = graycomatrix(channel, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=16, symmetric=True, normed=True)

      # Calculate properties of the GLCM
      contrast = np.mean(graycoprops(glcm, 'contrast'))
      dissimilarity = np.mean(graycoprops(glcm, 'dissimilarity'))
      homogeneity = np.mean(graycoprops(glcm, 'homogeneity')) #inverse difference moment
      energy = np.mean(graycoprops(glcm, 'energy')) #angular second moment
      correlation = np.mean(graycoprops(glcm, 'correlation'))
      glcm_dict[fullname+name+'contrast'] = contrast
      glcm_dict[fullname+name+'dissimilarity'] = dissimilarity
      glcm_dict[fullname+name+'homogeneity'] = homogeneity
      glcm_dict[fullname+name+'energy'] = energy
      glcm_dict[fullname+name+'correlation'] = correlation

      #variance
      variance = np.var(glcm.flatten())

      # Entropy
      #entropy = -np.sum(glcm * np.log(glcm + (glcm == 0)))
      entropy_v = -np.sum(glcm * np.log2(glcm + np.finfo(float).eps))#copilot

      #sum entropy
      sum_entropy = calculate_sum_entropy(glcm)

      # Sum Average
      sum_average = np.sum(np.outer(np.arange(2, 2 * len(glcm) + 2), glcm))

      # Sum Variance
      sum_variance = np.sum(np.outer((np.arange(2, 2 * len(glcm) + 2) - sum_average) ** 2, glcm))

      # Difference Variance
      diff_variance = np.sum(np.outer(np.arange(len(glcm)) ** 2, glcm))

      mcc_value = calculate_mcc(glcm)

      imc1_value = calculate_imc1(glcm)

      difference_entropy = calculate_difference_entropy(glcm)

      imc2_value = calculate_imc2pi(glcm)

      glcm_dict[fullname+name+'variance'] = variance
      glcm_dict[fullname+name+'entropy'] = entropy_v
      glcm_dict[fullname+name+'sum_entropy'] = sum_entropy
      glcm_dict[fullname+name+'sum_average'] = sum_average
      glcm_dict[fullname+name+'sum_variance'] = sum_variance
      glcm_dict[fullname+name+'diff_variance'] = diff_variance
      glcm_dict[fullname+name+'mcc_value'] = mcc_value
      glcm_dict[fullname+name+'imc1_value'] = imc1_value
      glcm_dict[fullname+name+'difference_entropy'] = difference_entropy
      glcm_dict[fullname+name+'imc2_value'] = imc2_value
  return glcm_dict

In [44]:
IMG_NUM = "1"
with open('RandomForestClassifier.pkl', 'rb') as file:
  loaded_model = pickle.load(file)

for folder in folder_names:
  img_path = None
  mask_path = None
  for file_name in os.listdir(os.path.join(root_folder, folder)):
    # If a file is an image and has 'mask' in its name, save its path
    if fnmatch.fnmatch(file_name, '*mask*.jpg') or fnmatch.fnmatch(file_name, '*mask*.png'):
      print(f"Found image with 'mask' in name: {file_name}")
      mask_path = os.path.join(root_folder, folder, file_name)
    # If there's an image file named '1' (with any extension), save its path
    if fnmatch.fnmatch(file_name, '1.*'):
      print(f"Found image file named '1': {file_name}")
      img_path = os.path.join(root_folder, folder, file_name)

  if(img_path != None and mask_path != None):
    dds = process(img_path, mask_path)
    dds = dds[dds.columns.drop(list(dds.filter(regex='21')))]#sometimes slic will fail to generate 20, this is nuclear solution
    dds.fillna(0, inplace=True)
    dds = dds.replace([np.inf, -np.inf], np.finfo(np.float32).max)
    dds = dds.where(dds <= np.finfo(np.float32).max, np.finfo(np.float32).max)
    y_test = loaded_model.predict(dds)
    print(folder + " is " + str(y_test))

Found image file named '1': 1.png
Found image with 'mask' in name: ISIC_5117966_mask.png
['test image is Lentigo maligna']


['test image is Lentigo maligna']
